# Clustering for gevtev

In [2]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from sklearn.neighbors import DistanceMetric
from sklearn.metrics.pairwise import pairwise_distances
from sklearn import cluster
from sklearn.decomposition.pca import PCA
from sklearn.manifold import TSNE
import seaborn as sns
from definitions import *
from utils import *

In [3]:
#path_to_gamma_bins shoud contain an absolute path to your gamma-bins directory
path_to_gamma_bins = os.path.dirname(os.getcwd()) + "/"

In [4]:
#name columns included in the final display
other_names = ['gev_1FGL_Name', 'gev_CLASS1', 'tev_fermi_names', 'tev_classes', 's_MAIN_ID', 's_class','s_simbad', 'marked'] + list_xmm_other_columns()
#from gamma bins directory
path_to_catalog = "markeddata/" + get_name_for_gevtevxmm() + "_normalized.txt"

In [5]:
spectra = pd.read_csv(path_to_gamma_bins + path_to_catalog, sep='\t')
print(path_to_gamma_bins +path_to_catalog)
spectra.head()

/home/masha/PycharmProjects/gamma-bins/markeddata/gevtevxmm_simbadclasses_06_normalized.txt


,xmm_SC_EP_1_FLUX,xmm_SC_EP_2_FLUX,xmm_SC_EP_3_FLUX,xmm_SC_EP_4_FLUX,xmm_SC_EP_5_FLUX,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,...,gev_GLAT,gev_RAJ2000,gev_DEJ2000,tev_fermi_names,tev_classes,tev_glat,tev_glon,tev_pos_dec,tev_pos_ra,marked
0,0.068553,0.061488,0.059425,0.059511,0.062076,0.067910,0.067770,0.075313,0.069360,0.066077,...,1.337755,6.4326,64.069298,b'',b'snr',1.41293,120.092361,64.140802,6.339723,0
1,0.068907,0.061626,0.059742,0.060119,0.063590,0.067621,0.067482,0.074993,0.069065,0.065796,...,1.337755,6.4326,64.069298,b'',b'snr',1.41293,120.092361,64.140802,6.339723,0
2,0.081538,0.073453,0.073100,0.074908,-inf,0.068704,0.068562,0.076194,0.070171,0.066850,...,1.337755,6.4326,64.069298,b'',b'snr',1.41293,120.092361,64.140802,6.339723,0
3,0.072884,0.066512,0.065242,0.065769,0.076760,0.064350,0.064217,0.071365,0.065725,0.062614,...,1.337755,6.4326,64.069298,b'',b'snr',1.41293,120.092361,64.140802,6.339723,0
4,0.066945,0.059547,0.057954,0.058218,0.062214,0.068519,0.068377,0.075988,0.069982,0.066670,...,1.337755,6.4326,64.069298,b'',b'snr',1.41293,120.092361,64.140802,6.339723,0


In [6]:
spectra = spectra.copy()
spectra = spectra.reset_index(drop=True)

In [7]:
spectrum_columns = list_gev_spectrum_columns() + list_tev_spectrum_columns()

In [8]:
only_spectra = spectra[spectrum_columns].copy()

In [9]:
matrix_inf = (only_spectra != -np.inf)

In [15]:
matrix_inf

,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,tev_0.3TeV,tev_1TeV,tev_3TeV,tev_10TeV,tev_30TeV
0,True,True,True,True,True,False,True,True,True,True,True
1,True,True,True,True,True,False,True,True,True,True,True
2,True,True,True,True,True,False,True,True,True,True,True
3,True,True,True,True,True,False,True,True,True,True,True
4,True,True,True,True,True,False,True,True,True,True,True
5,True,True,True,True,True,False,True,True,True,True,True
6,True,True,True,True,True,False,True,True,True,True,True
7,True,True,True,True,True,False,True,True,True,True,True
8,True,True,True,True,True,False,True,True,True,True,True
9,True,True,True,True,True,False,True,True,True,True,True


In [11]:
only_spectra

,gev_nuFnu10000_100000,gev_nuFnu1000_3000,gev_nuFnu100_300,gev_nuFnu3000_10000,gev_nuFnu300_1000,gev_nuFnu30_100,tev_0.3TeV,tev_1TeV,tev_3TeV,tev_10TeV,tev_30TeV
0,0.067910,0.067770,0.075313,0.069360,0.066077,-inf,0.066280,0.070908,0.073742,0.068283,0.063302
1,0.067621,0.067482,0.074993,0.069065,0.065796,-inf,0.065999,0.070606,0.073429,0.067993,0.063033
2,0.068704,0.068562,0.076194,0.070171,0.066850,-inf,0.067055,0.071737,0.074605,0.069082,0.064042
3,0.064350,0.064217,0.071365,0.065725,0.062614,-inf,0.062806,0.067191,0.069877,0.064704,0.059984
4,0.068519,0.068377,0.075988,0.069982,0.066670,-inf,0.066875,0.071543,0.074403,0.068896,0.063870
5,0.064035,0.063903,0.071016,0.065403,0.062307,-inf,0.062499,0.066862,0.069535,0.064388,0.059691
6,0.066680,0.066542,0.073949,0.068104,0.064880,-inf,0.065080,0.069623,0.072406,0.067047,0.062156
7,0.064007,0.063875,0.070985,0.065374,0.062280,-inf,0.062472,0.066833,0.069505,0.064359,0.059665
8,0.063906,0.063774,0.070873,0.065271,0.062181,-inf,0.062373,0.066727,0.069395,0.064258,0.059570
9,0.063735,0.063603,0.070683,0.065096,0.062015,-inf,0.062206,0.066548,0.069209,0.064085,0.059411


In [7]:
def get_distance_metric(metric_function):
    if type(metric_function) == str:
        dist = DistanceMetric.get_metric(metric_function)
    else:
        dist = DistanceMetric.get_metric("pyfunc", func=metric_function)
    return dist

In [8]:
from matplotlib import colors as mcolors


colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)

# Sort colors by hue, saturation, value and name.
by_hsv = sorted((tuple(mcolors.rgb_to_hsv(mcolors.to_rgba(color)[:3])), name)
                for name, color in colors.items())
sorted_names = [name for hsv, name in by_hsv]
colors = list(colors.values())[19:]

* ### earth mover's distance

In [9]:
# earth mover's distance
metric_function = st.wasserstein_distance
spectra_distances = pairwise_distances(spectra[spectrum_columns], metric=metric_function, n_jobs=6)
# spectra_distances = get_distance_metric(metric_function).pairwise(spectra[spectrum_columns])
#pd.DataFrame(spectra_distances).to_csv(path_to_gamma_bins +"markeddata/gevtev_spectra_EM_distances.txt", index=False)

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/home/masha/anaconda3/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/home/masha/anaconda3/lib/pyth

In [ ]:
from utils import *

In [ ]:
spectra_2D = build_images_KMeans(spectra, spectrum_columns, spectra_distances, colors, TSNE_learning_rate=300, TSNE_n_iter=340, TSNE_learning_rate2=200)

In [ ]:
list_spectra_clusters, centers = print_clusters_structure_KMeans(spectra, spectrum_columns, other_names, spectra_distances, 6, colors,spectra_2D)
for i in range(len(list_spectra_clusters)):
    print("{} cluster ({})".format(i, colors[i]))
    display(list_spectra_clusters[i])
print("centroids:")
display(centers)

In [ ]:
spectra_2D = build_images_DBSCAN(spectra, spectrum_columns, spectra_distances, colors, TSNE_learning_rate=500, TSNE_n_iter=1500, TSNE_learning_rate2=300)

In [ ]:
list_spectra_clusters, centers = print_clusters_structure_DBSCAN(spectra, spectrum_columns, other_names, spectra_distances, 0.06, colors, spectra_2D)
for i in range(len(list_spectra_clusters)):
    print("{} cluster ({})".format(i, colors[i]))
    display(list_spectra_clusters[i])
print("centroids:")
display(centers)

In [ ]:
spectra_distances.min(), spectra_distances.max()

* ### euclidean distance

In [ ]:
# euclidean distance
from scipy.spatial.distance import euclidean
metric_function = "euclidean"
spectra_distances = pairwise_distances(spectra[spectrum_columns], metric=metric_function)

In [ ]:
spectra_distances.min(), spectra_distances.max()

In [ ]:
spectra_2D = build_images_KMeans(spectra, spectrum_columns, spectra_distances, colors, TSNE_learning_rate=500, TSNE_n_iter=1500, TSNE_learning_rate2=300)
list_spectra_clusters, centers = print_clusters_structure_KMeans(spectra, spectrum_columns, other_names, spectra_distances, 6, colors, spectra_2D)
for i in range(len(list_spectra_clusters)):
    print("{} cluster ({})".format(i, colors[i]))
    display(list_spectra_clusters[i])
print("centroids:")
display(centers)

In [ ]:
markers = ['^', 'o']
cols = spectra['marked'].apply(lambda x: colors[x])
col = spectra['marked']

plt.subplots(figsize=(18, 6))
plt.subplot(131)
plt.title("PCA")
for i in range(len(spectra_2D)):
        #print(i)
        #print(spectra_2D[i, 0])
        #print(spectra_2D[i, 1])
        #print(cols[i])
        #print(col[i])
        if col[i] == 0:
            markersize = 5
        else:
            markersize = 100
        plt.scatter(spectra_2D[i, 0], spectra_2D[i, 1], c=cols[i], alpha=0.5, marker=markers[col[i]], s=markersize)

In [ ]:
eps_l = [0.1 * i for i in range(12, 0, -2)] + [0.1]
print(eps_l)
spectra_2D = build_images_DBSCAN(spectra, spectrum_columns, spectra_distances, colors, eps_l=eps_l, TSNE_learning_rate=1000, TSNE_n_iter=1500, TSNE_learning_rate2=900)
list_spectra_clusters, centers = print_clusters_structure_DBSCAN(spectra, spectrum_columns, other_names, spectra_distances, 0.6, colors, spectra_2D)
for i in range(len(list_spectra_clusters)):
    print("{} cluster ({})".format(i, colors[i]))
    display(list_spectra_clusters[i])
print("centroids:")
display(centers)

* ### cosine distance

In [ ]:
# cosine distance
# from scipy.spatial.distance import cosine
metric_function = "cosine"
spectra_distances = pairwise_distances(spectra[spectrum_columns], metric=metric_function)

In [ ]:
spectra_2D = build_images_KMeans(spectra, spectrum_columns, spectra_distances, colors, TSNE_learning_rate=500, TSNE_n_iter=1500, TSNE_learning_rate2=300)
list_spectra_clusters, centers = print_clusters_structure_KMeans(spectra, spectrum_columns, other_names, spectra_distances, 5, colors, spectra_2D)
for i in range(len(list_spectra_clusters)):
    print("{} cluster ({})".format(i, colors[i]))
    display(list_spectra_clusters[i])
print("centroids:")
display(centers)

In [ ]:
eps_l = [0.1 * i for i in range(10, 0, -2)] + [0.1]
print(eps_l)
spectra_2D = build_images_DBSCAN(spectra, spectrum_columns, spectra_distances, colors, eps_l=eps_l, TSNE_learning_rate=1000, TSNE_n_iter=1500, TSNE_learning_rate2=900)
list_spectra_clusters, centers = print_clusters_structure_DBSCAN(spectra, spectrum_columns, other_names, spectra_distances, 0.6, colors, spectra_2D)
for i in range(len(list_spectra_clusters)):
    print("{} cluster ({})".format(i, colors[i]))
    display(list_spectra_clusters[i])
print("centroids:")
display(centers)

* ### energy distance

In [ ]:
# energy distance
metric_function = st.energy_distance
spectra_distances = pairwise_distances(spectra[spectrum_columns], metric=metric_function)

In [ ]:
spectra_2D  = build_images_KMeans(spectra, spectrum_columns, spectra_distances, colors, TSNE_learning_rate=500, TSNE_n_iter=1500, TSNE_learning_rate2=300)
list_spectra_clusters, centers = print_clusters_structure_KMeans(spectra, spectrum_columns, other_names, spectra_distances, 4, colors, spectra_2D)
for i in range(len(list_spectra_clusters)):
    print("{} cluster ({})".format(i, colors[i]))
    display(list_spectra_clusters[i])
print("centroids:")
display(centers)

In [ ]:
eps_l = [0.01 * i for i in range(20, 0, -3)]
print(eps_l)
spectra_2D = build_images_DBSCAN(spectra, spectrum_columns, spectra_distances, colors, eps_l=eps_l, TSNE_learning_rate=1000, TSNE_n_iter=1500, TSNE_learning_rate2=900)
list_spectra_clusters, centers = print_clusters_structure_DBSCAN(spectra, spectrum_columns, other_names, spectra_distances, 0.17, colors, spectra_2D)
for i in range(len(list_spectra_clusters)):
    print("{} cluster ({})".format(i, colors[i]))
    display(list_spectra_clusters[i])
print("centroids:")
display(centers)